<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
from bs4 import BeautifulSoup
import requests

from IPython.core.display import display, HTML

import pandas as pd

import re

import numpy as np

import datetime

In [2]:
#Retrieve breakdown of games from october to end of march

urlList = [
    'https://www.basketball-reference.com/leagues/NBA_2015_games-october.html',
    'https://www.basketball-reference.com/leagues/NBA_2015_games-november.html',
    'https://www.basketball-reference.com/leagues/NBA_2015_games-december.html',
    'https://www.basketball-reference.com/leagues/NBA_2015_games-january.html',
    'https://www.basketball-reference.com/leagues/NBA_2015_games-february.html',
    'https://www.basketball-reference.com/leagues/NBA_2015_games-march.html'
]
soupList = []
for url in urlList:
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    soupList.append(soup)
[soup1,soup2,soup3,soup4,soup5,soup6] = soupList

In [3]:
headerData = soup1.find(class_ = 'overthrow table_container').find_all('tr')[0].find_all('th')

In [4]:
column_headers = []

#get headers
for header in headerData:
    column_headers.append(header['data-stat'])
column_headers

['date_game',
 'game_start_time',
 'visitor_team_name',
 'visitor_pts',
 'home_team_name',
 'home_pts',
 'box_score_text',
 'overtimes',
 'attendance',
 'game_remarks']

In [5]:
fullDataList = []

for soup in soupList:
    cellData = soup.find(class_ = 'overthrow table_container').find_all('tr')
    for row in range(1,len(cellData)):
        rowList = []
        for cell in cellData[row].find_all(lambda tag: tag.name == 'td' or tag.name == 'th'):
            if cell.contents == []:
                value = ''
            elif cell.contents[0].name == 'a':
                if cell.contents[0].contents[0] == 'Box Score':
                    value = 'https://www.basketball-reference.com'+ cell.contents[0]['href']
                else:
                    value = cell.contents[0].contents[0]
            else:
                value = cell.contents[0]
            rowList.append(value)
        fullDataList.append(rowList)

cleanCellData = [x for x in fullDataList if x != []]

cleanCellData

[['Tue, Oct 28, 2014',
  '8:00p',
  'Orlando Magic',
  '84',
  'New Orleans Pelicans',
  '101',
  'https://www.basketball-reference.com/boxscores/201410280NOP.html',
  '',
  '17,097',
  ''],
 ['Tue, Oct 28, 2014',
  '8:00p',
  'Dallas Mavericks',
  '100',
  'San Antonio Spurs',
  '101',
  'https://www.basketball-reference.com/boxscores/201410280SAS.html',
  '',
  '19,615',
  ''],
 ['Tue, Oct 28, 2014',
  '10:30p',
  'Houston Rockets',
  '108',
  'Los Angeles Lakers',
  '90',
  'https://www.basketball-reference.com/boxscores/201410280LAL.html',
  '',
  '18,997',
  ''],
 ['Wed, Oct 29, 2014',
  '7:00p',
  'Milwaukee Bucks',
  '106',
  'Charlotte Hornets',
  '108',
  'https://www.basketball-reference.com/boxscores/201410290CHO.html',
  'OT',
  '19,439',
  ''],
 ['Wed, Oct 29, 2014',
  '7:00p',
  'Philadelphia 76ers',
  '91',
  'Indiana Pacers',
  '103',
  'https://www.basketball-reference.com/boxscores/201410290IND.html',
  '',
  '18,165',
  ''],
 ['Wed, Oct 29, 2014',
  '7:30p',
  'Atlan

In [6]:
#Combine Header and Data to create intiial DF

df = pd.DataFrame(cleanCellData,columns = column_headers)

In [7]:
def date_change(row):
    t = datetime.datetime.strptime(row['date_game'].replace(',',''), "%a %b %d %Y")
    convert_date = t.strftime('%m/%d/%Y')
    return(convert_date)

df['Date'] = pd.to_datetime(df.apply(date_change,axis = 1))

In [8]:
nbaDict = {
'Atlanta Hawks': 'ATL',
'Brooklyn Nets': 'BRK',
'Boston Celtics': 'BOS',
'Charlotte Hornets': 'CHO',
'Chicago Bulls': 'CHI',
'Cleveland Cavaliers': 'CLE',
'Dallas Mavericks': 'DAL',
'Denver Nuggets': 'DEN',
'Detroit Pistons': 'DET',
'Golden State Warriors': 'GSW',
'Houston Rockets': 'HOU',
'Indiana Pacers': 'IND',
'Los Angeles Clippers': 'LAC',
'Los Angeles Lakers': 'LAL',
'Memphis Grizzlies': 'MEM',
'Miami Heat': 'MIA',
'Milwaukee Bucks': 'MIL',
'Minnesota Timberwolves': 'MIN',
'New Orleans Pelicans': 'NOP',
'New York Knicks': 'NYK',
'Oklahoma City Thunder': 'OKC',
'Orlando Magic': 'ORL',
'Philadelphia 76ers': 'PHI',
'Phoenix Suns': 'PHO',
'Portland Trail Blazers': 'POR',
'Sacramento Kings': 'SAC',
'San Antonio Spurs': 'SAS',
'Toronto Raptors': 'TOR',
'Utah Jazz': 'UTA',
'Washington Wizards': 'WAS'}

In [9]:
statColumns = ['Team TS%','Team eFG%','Team ORtg','Opp TS%','Opp eFG%','Opp ORtg']

In [10]:
def statRecorder(row):
    url1 = row['box_score_text']
    response1 = requests.get(url1)
    page1 = response1.text
    soup1 = BeautifulSoup(page1, "lxml")
    team_list = [nbaDict[row['visitor_team_name']],nbaDict[row['home_team_name']]]
    print([row['visitor_team_name'],row['home_team_name']])
    statsList = []
    index = [1,2,-2]
    for team in team_list:
        boxScoreLink = "box-{}-game-advanced".format(team)
        totalStats = soup1.find('table', id =boxScoreLink).find_all('tr')[-1].find_all('td')
        for i in index:
            statsList.append(totalStats[i].contents[0])
    return(statsList)

In [11]:
#Create new stats list incrementally and then concatenate after
stats_list = []

In [12]:
firstBatch = df[0:1200].apply(statRecorder,axis = 1)
for i in firstBatch:
    stats_list.append(i)

['Orlando Magic', 'New Orleans Pelicans']
['Dallas Mavericks', 'San Antonio Spurs']
['Houston Rockets', 'Los Angeles Lakers']
['Milwaukee Bucks', 'Charlotte Hornets']
['Philadelphia 76ers', 'Indiana Pacers']
['Atlanta Hawks', 'Toronto Raptors']
['Washington Wizards', 'Miami Heat']
['Brooklyn Nets', 'Boston Celtics']
['Minnesota Timberwolves', 'Memphis Grizzlies']
['Chicago Bulls', 'New York Knicks']
['Detroit Pistons', 'Denver Nuggets']
['Houston Rockets', 'Utah Jazz']
['Golden State Warriors', 'Sacramento Kings']
['Los Angeles Lakers', 'Phoenix Suns']
['Oklahoma City Thunder', 'Portland Trail Blazers']
['Washington Wizards', 'Orlando Magic']
['New York Knicks', 'Cleveland Cavaliers']
['Detroit Pistons', 'Minnesota Timberwolves']
['Utah Jazz', 'Dallas Mavericks']
['Oklahoma City Thunder', 'Los Angeles Clippers']
['Memphis Grizzlies', 'Indiana Pacers']
['Cleveland Cavaliers', 'Chicago Bulls']
['Philadelphia 76ers', 'Milwaukee Bucks']
['San Antonio Spurs', 'Phoenix Suns']
['Portland Trai

['Phoenix Suns', 'Toronto Raptors']
['New York Knicks', 'Houston Rockets']
['Indiana Pacers', 'Dallas Mavericks']
['Chicago Bulls', 'Utah Jazz']
['Atlanta Hawks', 'Washington Wizards']
['Golden State Warriors', 'Miami Heat']
['Detroit Pistons', 'Milwaukee Bucks']
['Sacramento Kings', 'New Orleans Pelicans']
['Chicago Bulls', 'Denver Nuggets']
['Portland Trail Blazers', 'Charlotte Hornets']
['Washington Wizards', 'Cleveland Cavaliers']
['Golden State Warriors', 'Orlando Magic']
['Brooklyn Nets', 'Philadelphia 76ers']
['Toronto Raptors', 'Atlanta Hawks']
['New York Knicks', 'Dallas Mavericks']
['Los Angeles Clippers', 'Detroit Pistons']
['Sacramento Kings', 'Houston Rockets']
['Milwaukee Bucks', 'Minnesota Timberwolves']
['Utah Jazz', 'Oklahoma City Thunder']
['Indiana Pacers', 'San Antonio Spurs']
['Denver Nuggets', 'Phoenix Suns']
['Memphis Grizzlies', 'Los Angeles Lakers']
['Chicago Bulls', 'Boston Celtics']
['Golden State Warriors', 'Charlotte Hornets']
['New Orleans Pelicans', 'Atla

['Memphis Grizzlies', 'Cleveland Cavaliers']
['Detroit Pistons', 'Brooklyn Nets']
['Boston Celtics', 'Miami Heat']
['Philadelphia 76ers', 'Orlando Magic']
['Los Angeles Lakers', 'Sacramento Kings']
['Phoenix Suns', 'Washington Wizards']
['Indiana Pacers', 'Minnesota Timberwolves']
['New Orleans Pelicans', 'Oklahoma City Thunder']
['Denver Nuggets', 'Charlotte Hornets']
['Toronto Raptors', 'Chicago Bulls']
['Portland Trail Blazers', 'Houston Rockets']
['Utah Jazz', 'Memphis Grizzlies']
['Atlanta Hawks', 'Dallas Mavericks']
['Los Angeles Clippers', 'San Antonio Spurs']
['Sacramento Kings', 'Golden State Warriors']
['Minnesota Timberwolves', 'Cleveland Cavaliers']
['New Orleans Pelicans', 'Indiana Pacers']
['Boston Celtics', 'Orlando Magic']
['Chicago Bulls', 'Washington Wizards']
['Los Angeles Clippers', 'Atlanta Hawks']
['Denver Nuggets', 'Brooklyn Nets']
['Philadelphia 76ers', 'Miami Heat']
['Charlotte Hornets', 'Milwaukee Bucks']
['Portland Trail Blazers', 'Oklahoma City Thunder']
['D

['Philadelphia 76ers', 'Detroit Pistons']
['Washington Wizards', 'Brooklyn Nets']
['Atlanta Hawks', 'Chicago Bulls']
['Golden State Warriors', 'Houston Rockets']
['Minnesota Timberwolves', 'Denver Nuggets']
['Portland Trail Blazers', 'Memphis Grizzlies']
['Los Angeles Clippers', 'Sacramento Kings']
['New Orleans Pelicans', 'Toronto Raptors']
['Oklahoma City Thunder', 'Orlando Magic']
['Utah Jazz', 'San Antonio Spurs']
['Minnesota Timberwolves', 'Charlotte Hornets']
['Philadelphia 76ers', 'Washington Wizards']
['Detroit Pistons', 'Atlanta Hawks']
['Boston Celtics', 'Los Angeles Clippers']
['Denver Nuggets', 'Golden State Warriors']
['Dallas Mavericks', 'Memphis Grizzlies']
['Indiana Pacers', 'Houston Rockets']
['New Orleans Pelicans', 'New York Knicks']
['Chicago Bulls', 'Cleveland Cavaliers']
['Toronto Raptors', 'Milwaukee Bucks']
['Los Angeles Lakers', 'Phoenix Suns']
['Sacramento Kings', 'Portland Trail Blazers']
['Oklahoma City Thunder', 'Miami Heat']
['San Antonio Spurs', 'Denver N

['San Antonio Spurs', 'Los Angeles Clippers']
['New Orleans Pelicans', 'Orlando Magic']
['Indiana Pacers', 'Philadelphia 76ers']
['Toronto Raptors', 'Atlanta Hawks']
['Chicago Bulls', 'Detroit Pistons']
['Miami Heat', 'New York Knicks']
['Phoenix Suns', 'Minnesota Timberwolves']
['Cleveland Cavaliers', 'Washington Wizards']
['Houston Rockets', 'Dallas Mavericks']
['Denver Nuggets', 'Milwaukee Bucks']
['Portland Trail Blazers', 'Utah Jazz']
['Boston Celtics', 'Sacramento Kings']
['San Antonio Spurs', 'Golden State Warriors']
['Brooklyn Nets', 'Los Angeles Lakers']
['Oklahoma City Thunder', 'Charlotte Hornets']
['New Orleans Pelicans', 'Miami Heat']
['Phoenix Suns', 'Chicago Bulls']
['Toronto Raptors', 'Houston Rockets']
['Sacramento Kings', 'Los Angeles Clippers']
['Cleveland Cavaliers', 'New York Knicks']
['Washington Wizards', 'Detroit Pistons']
['Atlanta Hawks', 'Milwaukee Bucks']
['Golden State Warriors', 'Indiana Pacers']
['Philadelphia 76ers', 'Orlando Magic']
['Denver Nuggets', '

['San Antonio Spurs', 'New York Knicks']
['Orlando Magic', 'Houston Rockets']
['Milwaukee Bucks', 'New Orleans Pelicans']
['Charlotte Hornets', 'Los Angeles Clippers']
['Brooklyn Nets', 'Cleveland Cavaliers']
['Detroit Pistons', 'Philadelphia 76ers']
['Portland Trail Blazers', 'Miami Heat']
['Minnesota Timberwolves', 'Toronto Raptors']
['Indiana Pacers', 'Chicago Bulls']
['Orlando Magic', 'Dallas Mavericks']
['San Antonio Spurs', 'Milwaukee Bucks']
['Boston Celtics', 'Oklahoma City Thunder']
['Los Angeles Clippers', 'Sacramento Kings']
['Atlanta Hawks', 'Golden State Warriors']
['Washington Wizards', 'Utah Jazz']
['Minnesota Timberwolves', 'New York Knicks']
['Denver Nuggets', 'Houston Rockets']
['New Orleans Pelicans', 'Phoenix Suns']
['Utah Jazz', 'Los Angeles Lakers']
['Portland Trail Blazers', 'Orlando Magic']
['New York Knicks', 'Philadelphia 76ers']
['Milwaukee Bucks', 'Brooklyn Nets']
['Indiana Pacers', 'Cleveland Cavaliers']
['Denver Nuggets', 'Miami Heat']
['Toronto Raptors', 

In [13]:
statsdf = pd.DataFrame(stats_list,columns = statColumns)

In [14]:
resultdf = pd.concat([df, statsdf], axis=1)
resultdf.head()

,date_game,game_start_time,visitor_team_name,visitor_pts,home_team_name,home_pts,box_score_text,overtimes,attendance,game_remarks,Date,Team TS%,Team eFG%,Team ORtg,Opp TS%,Opp eFG%,Opp ORtg
0,"Tue, Oct 28, 2014",8:00p,Orlando Magic,84,New Orleans Pelicans,101,https://www.basketball-reference.com/boxscores...,,"17,097",,2014-10-28,.450,.405,88.2,.441,.426,106.1
1,"Tue, Oct 28, 2014",8:00p,Dallas Mavericks,100,San Antonio Spurs,101,https://www.basketball-reference.com/boxscores...,,"19,615",,2014-10-28,.579,.538,116.1,.656,.629,117.3
2,"Tue, Oct 28, 2014",10:30p,Houston Rockets,108,Los Angeles Lakers,90,https://www.basketball-reference.com/boxscores...,,"18,997",,2014-10-28,.568,.507,118.8,.468,.373,99.0
3,"Wed, Oct 29, 2014",7:00p,Milwaukee Bucks,106,Charlotte Hornets,108,https://www.basketball-reference.com/boxscores...,OT,"19,439",,2014-10-29,.585,.531,104.8,.475,.436,106.8
4,"Wed, Oct 29, 2014",7:00p,Philadelphia 76ers,91,Indiana Pacers,103,https://www.basketball-reference.com/boxscores...,,"18,165",,2014-10-29,.445,.416,96.7,.552,.519,109.5


In [15]:
#visitor stats
visitor_df = resultdf.iloc[:,np.r_[0:7,11:14]]
visitor_df.head()

#home stats
home_df = resultdf.iloc[:,np.r_[0:2,4:6,2:4,6,14:17]]
home_df.head()

visitor_df.head()


,date_game,game_start_time,visitor_team_name,visitor_pts,home_team_name,home_pts,box_score_text,Team TS%,Team eFG%,Team ORtg
0,"Tue, Oct 28, 2014",8:00p,Orlando Magic,84,New Orleans Pelicans,101,https://www.basketball-reference.com/boxscores...,.450,.405,88.2
1,"Tue, Oct 28, 2014",8:00p,Dallas Mavericks,100,San Antonio Spurs,101,https://www.basketball-reference.com/boxscores...,.579,.538,116.1
2,"Tue, Oct 28, 2014",10:30p,Houston Rockets,108,Los Angeles Lakers,90,https://www.basketball-reference.com/boxscores...,.568,.507,118.8
3,"Wed, Oct 29, 2014",7:00p,Milwaukee Bucks,106,Charlotte Hornets,108,https://www.basketball-reference.com/boxscores...,.585,.531,104.8
4,"Wed, Oct 29, 2014",7:00p,Philadelphia 76ers,91,Indiana Pacers,103,https://www.basketball-reference.com/boxscores...,.445,.416,96.7


In [16]:
home_df.columns = ['date_game','game_start_time','team_name','team_pts','opposing_team','opposing_pts','box_score','TS%','eFG%','ORtg']

visitor_df.columns = ['date_game','game_start_time','team_name','team_pts','opposing_team','opposing_pts','box_score','TS%','eFG%','ORtg']


In [17]:
frames = [visitor_df, home_df]
agg_data = pd.concat(frames,ignore_index=True)

In [18]:
def date_change(row):
    t = datetime.datetime.strptime(row['date_game'].replace(',',''), "%a %b %d %Y")
    convert_date = t.strftime('%m/%d/%Y')
    return(convert_date)

agg_data['Date'] = pd.to_datetime(agg_data.apply(date_change,axis = 1))

In [19]:
agg_data['date_game'] = agg_data['Date']
del agg_data['Date']

In [20]:
ordered_agg_data = agg_data.sort_values(by = ['team_name','date_game']).reset_index()

In [21]:
rolling_stat_data = agg_data.sort_values(by = ['team_name','date_game'])

testdata_grouped_rolling = rolling_stat_data.groupby('team_name')[['TS%','eFG%','ORtg']].rolling(window=30, min_periods=1).mean().reset_index()



In [22]:
del testdata_grouped_rolling['level_1']
del testdata_grouped_rolling['team_name']

In [23]:
testdata_grouped_rolling.columns = ['TS1%','eFG1%','ORtg1']

In [24]:
full_stats_data = pd.concat([ordered_agg_data ,testdata_grouped_rolling], axis=1)

In [25]:
#Shift average stats so each row has the average of the games previously
full_stats_data[['TS%','eFG%','ORtg']] = full_stats_data.groupby('team_name')['TS1%','eFG1%','ORtg1'].apply(lambda grp: grp.shift(1))

In [26]:
#Deleted non-shifted columns

full_stats_data = full_stats_data.drop(['TS1%','eFG1%','ORtg1'],axis =1)

In [27]:
full_stats_data_2 = full_stats_data

In [28]:
newDf = pd.merge(full_stats_data_2,full_stats_data,left_on = ['date_game','team_name'],right_on = ['date_game','opposing_team'])

In [29]:
dedupe_new_df = newDf.drop_duplicates(subset = 'box_score_x',keep ='first')

In [30]:
dedupe_new_df.head()

,index_x,date_game,game_start_time_x,team_name_x,team_pts_x,opposing_team_x,opposing_pts_x,box_score_x,TS%_x,eFG%_x,...,index_y,game_start_time_y,team_name_y,team_pts_y,opposing_team_y,opposing_pts_y,box_score_y,TS%_y,eFG%_y,ORtg_y
0,5,2014-10-29,7:30p,Atlanta Hawks,102,Toronto Raptors,109,https://www.basketball-reference.com/boxscores...,NaN,NaN,...,1115,7:30p,Toronto Raptors,109,Atlanta Hawks,102,https://www.basketball-reference.com/boxscores...,NaN,NaN,NaN
1,1141,2014-11-01,7:30p,Atlanta Hawks,102,Indiana Pacers,92,https://www.basketball-reference.com/boxscores...,0.583000,0.5810,...,31,7:30p,Indiana Pacers,92,Atlanta Hawks,102,https://www.basketball-reference.com/boxscores...,0.550500,0.516500,105.850000
2,63,2014-11-05,8:30p,Atlanta Hawks,92,San Antonio Spurs,94,https://www.basketball-reference.com/boxscores...,0.597000,0.5695,...,1173,8:30p,San Antonio Spurs,94,Atlanta Hawks,92,https://www.basketball-reference.com/boxscores...,0.580500,0.545500,106.750000
3,70,2014-11-07,7:00p,Atlanta Hawks,119,Charlotte Hornets,122,https://www.basketball-reference.com/boxscores...,0.556333,0.5320,...,1180,7:00p,Charlotte Hornets,122,Atlanta Hawks,119,https://www.basketball-reference.com/boxscores...,0.501400,0.459200,99.420000
4,1193,2014-11-08,7:30p,Atlanta Hawks,103,New York Knicks,96,https://www.basketball-reference.com/boxscores...,0.559750,0.5320,...,83,7:30p,New York Knicks,96,Atlanta Hawks,103,https://www.basketball-reference.com/boxscores...,0.513167,0.478667,104.066667


In [31]:
dedupe_new_df.columns

Index(['index_x', 'date_game', 'game_start_time_x', 'team_name_x',
       'team_pts_x', 'opposing_team_x', 'opposing_pts_x', 'box_score_x',
       'TS%_x', 'eFG%_x', 'ORtg_x', 'index_y', 'game_start_time_y',
       'team_name_y', 'team_pts_y', 'opposing_team_y', 'opposing_pts_y',
       'box_score_y', 'TS%_y', 'eFG%_y', 'ORtg_y'],
      dtype='object')

In [32]:
allowed_df = dedupe_new_df[['date_game','team_name_x','opposing_team_x','TS%_x','eFG%_x', 'ORtg_x','TS%_y', 'eFG%_y', 'ORtg_y']]

allowed_df.columns = ['date_game','team_name','opposing_team','Team TS%','Team eFG%','Team ORtg','Opp TS%', 'Opp eFG%', 'Opp ORtg']

In [33]:
export_csv = allowed_df.to_csv (r'C:\Users\jeromerufin\Desktop\Metis\advstats_14_15.csv') #Don't forget to add '.csv' at the end of the path